In [3]:
import boto3
from io import BytesIO
from langchain_community.llms import Bedrock
from langchain_aws import ChatBedrock
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
session = boto3.Session(profile_name='my-session-1')
client = boto3.client('bedrock-runtime',region_name='ap-south-1',aws_access_key_id=session.get_credentials().access_key,
    aws_secret_access_key=session.get_credentials().secret_key,
    aws_session_token=session.get_credentials().token)
# Initialize S3 client
# s3 = boto3.client('s3')
s3 = session.resource('s3')

In [6]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = session.get_credentials().access_key
os.environ["AWS_SECRET_ACCESS_KEY"] = session.get_credentials().secret_key
os.environ["AWS_SESSION_TOKEN"] = session.get_credentials().token


In [7]:
for bucket in s3.buckets.all():
    print(bucket.name)

1-dataset-team-impulse
ai-alchemists-1729
ai-lifeline
ai-tigercare-lab
aigladiators
aws-glue-assets-011528265986-ap-south-1
byteforge-bkt
code-buddies
csk-telemetry-data
databricks-workspace-stack-e024d-bucket
databricks-workspace-stack-e024d-lambdazipsbucket-auo8rmzffvib
dataset-team-impulse
disruptops-healthcare
fault-squad
finops-bucket-807431
histreamics-uc-metastore-ap-south-1
insightsrxdata
iottelemetry1
iottestbucket01
mcd-force
medical-docs-upload
noname-tpl
ragents
shubham-aws-bedrock-practise1
sophos-optix-flowlogs-011528265986-ap-south-1
sophos-optix-flowlogs-011528265986-eu-west-1
sophos-optix-flowlogs-011528265986-us-east-1
sophos-optix-flowlogs-011528265986-us-east-2
teamfalcon
teamindia-aps-2025
teamindia-s3-aps1
tiger-tfg-poc
tiger-tpl-curebot-healthcare
tigermd-case3-tpl
tpl-byte-wizards
tpl-insightrx
tpl-mediquery
tpl-streamnova
tplhackathondata
vita-agents


In [8]:


llm = ChatBedrock(
    region_name="ap-south-1",
    # model_id="meta.llama3-70b-instruct-v1:0",
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    # inferenceConfiguration={
    #     "inferenceConfigurationId": "arn:aws:sts::011528265986:assumed-role/tpl-sandbox-batch/"
    # },
    client=client,
    # session=session,

)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""Use the following metadata to answer the question:
    {context}
    Question: {question}
    Answer:"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=column_retriever,  # or table_retriever
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

response = qa_chain.invoke({"query": "How long do patients wait in the emergency department?"})
print(response['result'])


# llm.invoke("generate a sql query for list the patients from patients table")

NameError: name 'column_retriever' is not defined

In [4]:
from langchain_community.retrievers.kendra import AmazonKendraRetriever
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

kendra_retriever = AmazonKendraRetriever(index_id="711df937-7bfb-4643-8e47-c433516d562d",min_score_confidence=0.1,top_k=10)

kendra_prompt = """
You are a medical data assistant helping retrieve structured answers from enterprise medical documents.
input metrics:
{question}
{context}

Given a **user query**, extract and return the following:

1. **Expected Metrics Returned**: Identify all the relevant column name(s) of the metric.
2. **Source Table**: Identify the relevant table name(s) that contains the above selected column(s).
3. **Description**: Provide a brief plain-language explanation of what this query is asking.

Format your response as JSON with the following keys:
expected_metrics
source_table
description
"""
prompt = PromptTemplate(
    template=kendra_prompt,
    input_variables=["question", "context"],
    # partial_variables={"role": "manager"}
)
llm = ChatBedrock(
    region_name="ap-south-1",
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=client,
)
retrieval_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=kendra_retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True,
    chain_type="stuff",  # or "map_reduce" for larger contexts
)

response = retrieval_chain.invoke({"query": "How long do patients usually wait before getting treated in the emergency room in a month?"})
print(response)

AccessDeniedException: An error occurred (AccessDeniedException) when calling the Retrieve operation: User: arn:aws:iam::207212954385:user/kishoresai.ganes@tigeranalytics.com is not authorized to perform: kendra:Retrieve on resource: arn:aws:kendra:ap-south-1:207212954385:index/711df937-7bfb-4643-8e47-c433516d562d because no identity-based policy allows the kendra:Retrieve action

In [ ]:
print(response['result'])  # This will contain the JSON response with intent, metric, access_granted, denied_reason

{
  "expected_metrics": ["avg_waiting_time_per_month"],
  "source_table": "emergency_metrics",
  "description": "This query is asking for the average amount of time patients have to wait before receiving treatment in the emergency room over the course of a month."
}


In [117]:
print(kendra_retriever.invoke("hospital"))

[Document(metadata={'type': 'DOCUMENT', 'result_id': '4a0c4946-c4e2-489d-a345-728ba0f919f1-ec5bcfff-82a3-409a-8274-99cda5560f03', 'document_id': 's3://tpl-insightrx/metadata/non_clinical_metadata.json', 'source': 'https://tpl-insightrx.s3.ap-south-1.amazonaws.com/metadata/non_clinical_metadata.json', 'title': 'non_clinical_metadata.json', 'excerpt': '...type": "Identifier",\n        "table_name": "hospital_administration_metrics",\n        "column_name": "hospital_id",\n        "data_type": "object",\n        "granularity": "Hospital",\n        "calculation": "None",\n        "description": "Unique identifier for the hospital or department...', 'document_attributes': {'_source_uri': 'https://tpl-insightrx.s3.ap-south-1.amazonaws.com/metadata/non_clinical_metadata.json', 's3_document_id': 'non_clinical_metadata.json'}, 'score': 'MEDIUM'}, page_content='Document Title: non_clinical_metadata.json\nDocument Excerpt: \ntype": "Identifier", "table_name": "hospital_administration_metrics", "c

In [ ]:


# Bucket and object key
bucket_name = 'tpl-insightrx'
object_key = 'HealthCare_Gold_datasets/Context/full_role_based_metric_access.json'

# Read the file into memory
response = s3.get_object(Bucket=bucket_name, Key=object_key)
file_content = response['Body'].read()

# If it's a CSV and you want to load into pandas:
df = pd.read_csv(BytesIO(file_content))
print(df.head())

NoCredentialsError: Unable to locate credentials

In [6]:
!export LANCE_URI="sk_T75W5VQSY5GGBFBS4IBU4FIYAIZLSTIV5Q2JLCPRZLOUKZC7XNRA===="

In [8]:
import json
import os

import pandas as pd
from langchain_community.vectorstores.lancedb import LanceDB
from langchain_openai.embeddings.azure import AzureOpenAIEmbeddings


LANCE_URI = os.environ.get("LANCE_URI", "sk_T75W5VQSY5GGBFBS4IBU4FIYAIZLSTIV5Q2JLCPRZLOUKZC7XNRA====")


In [ ]:



def generate_column_docs(meta_df):
    docs = []
    doc_metadata = []
    for _, row in meta_df.iterrows():
        data = {
            "table": row["table"],
            "column": row["column"],
            "datatype": row["column_datatype"],
            "description": row["column_description"],
        }
        doc = list(data.values())

        if row["primary_key"]:
            data["primary_key"] = row["primary_key"]
            doc.append("PRIMARY KEY")
        if not pd.isnull(row["foreign_key_ref"]):
            data["foreign_key_ref"] = row["foreign_key_ref"]
            doc.append(f"FOREIGN KEY REFERENCES {row['foreign_key_ref']}")

        doc_metadata.append(data)
        docs.append(" ".join(doc))

    return docs, doc_metadata


def generate_table_docs(meta_df):
    docs = []
    doc_metadata = []
    for _, row in meta_df.iterrows():
        data = {
            "table": row["table"],
            "description": row["table_description"],
        }
        doc = list(data.values())
        docs.append(" ".join(doc))
    return docs, doc_metadata

def populate_vector_store(metadata, lance_uri,TABLE_NAME):
    embeddings = AzureOpenAIEmbeddings(
        azure_endpoint=os.environ["EMBEDDING_AZURE_ENDPOINT"],
        azure_deployment=os.environ["EMBEDDING_AZURE_DEPLOYMENT"],
        api_version=os.environ["EMBEDDING_AZURE_API_VERSION"],
        openai_api_type="azure",
        model=os.environ["EMBEDDING_AZURE_MODEL"],
        api_key=os.environ["EMBEDDING_AZURE_API_KEY"],
    )

    schema_docs, doc_metadata = generate_column_docs(metadata)
    vector_store = LanceDB(uri=lance_uri, embedding=embeddings, table_name=TABLE_NAME)

    _ = vector_store.add_texts(texts=schema_docs, metadatas=doc_metadata)

    return vector_store

def populate_table_vector_store(metadata, lance_uri,TABLE_NAME):
    embeddings = AzureOpenAIEmbeddings(
        azure_endpoint=os.environ["EMBEDDING_AZURE_ENDPOINT"],
        azure_deployment=os.environ["EMBEDDING_AZURE_DEPLOYMENT"],
        api_version=os.environ["EMBEDDING_AZURE_API_VERSION"],
        openai_api_type="azure",
        model=os.environ["EMBEDDING_AZURE_MODEL"],
        api_key=os.environ["EMBEDDING_AZURE_API_KEY"],
    )

    schema_docs, doc_metadata = generate_table_docs(metadata)
    vector_store = LanceDB(uri=lance_uri, embedding=embeddings, table_name=TABLE_NAME)

    _ = vector_store.add_texts(texts=schema_docs, metadatas=doc_metadata)

    return vector_store